In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Amyotrophic_Lateral_Sclerosis"
cohort = "GSE95810"

# Input paths
in_trait_dir = "../../input/GEO/Amyotrophic_Lateral_Sclerosis"
in_cohort_dir = "../../input/GEO/Amyotrophic_Lateral_Sclerosis/GSE95810"

# Output paths
out_data_file = "../../output/preprocess/Amyotrophic_Lateral_Sclerosis/GSE95810.csv"
out_gene_data_file = "../../output/preprocess/Amyotrophic_Lateral_Sclerosis/gene_data/GSE95810.csv"
out_clinical_data_file = "../../output/preprocess/Amyotrophic_Lateral_Sclerosis/clinical_data/GSE95810.csv"
json_path = "../../output/preprocess/Amyotrophic_Lateral_Sclerosis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression from iPS derived neurons exposed to plasma from Alzheimer's (AD), pre-symptomatic AD, or control patients."
!Series_summary	"We have established proof of principle for the Indicator Cell Assay Platformé (iCAPé), a broadly applicable tool for blood-based diagnostics that uses specifically-selected, standardized cells as biosensors, relying on their innate ability to integrate and respond to diverse signals present in patientsÕ blood. To develop an assay, indicator cells are exposed in vitro to serum from case or control subjects and their global differential response patterns are used to train reliable, cost-effective disease classifiers based on a small number of features. In a feasibility study, the iCAP detected pre-symptomatic disease in a murine model of amyotrophic lateral sclerosis (ALS) with 94% accuracy (p-Value=3.81E-6) and correctly identified samples from a murine HuntingtonÕs disease model as non-carriers of ALS. In a p

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Determine if gene expression data is available
# Based on the background information, this appears to be a gene expression dataset from iPS derived neurons
is_gene_available = True

# 2. Check if trait, age, and gender data are available
# From the sample characteristics, we don't directly see trait (ALS status), age, or gender information

# 2.1 Trait Availability
# The background information mentions this is a study about Alzheimer's disease, not ALS
# Alzheimer's patients (18), pre-symptomatic Alzheimer's (20), and control patients (20)
# Since this study is about Alzheimer's, not ALS (Amyotrophic Lateral Sclerosis), trait data is not available
trait_row = None

# 2.2 Age Availability
# No age information in the sample characteristics
age_row = None

# 2.3 Gender Availability
# No gender information in the sample characteristics
gender_row = None

# Define conversion functions (even though we won't use them in this case)
def convert_trait(value):
    # If we had trait data, we would convert here
    if value is None:
        return None
    
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert the value to binary (0 for control, 1 for ALS)
    if 'control' in value.lower():
        return 0
    elif 'als' in value.lower() or 'amyotrophic lateral sclerosis' in value.lower():
        return 1
    else:
        return None

def convert_age(value):
    # If we had age data, we would convert here
    if value is None:
        return None
    
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Try to convert to float for continuous age
    try:
        return float(value)
    except:
        return None

def convert_gender(value):
    # If we had gender data, we would convert here
    if value is None:
        return None
    
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary (0 for female, 1 for male)
    value = value.lower()
    if 'female' in value or 'f' == value:
        return 0
    elif 'male' in value or 'm' == value:
        return 1
    else:
        return None

# 3. Save metadata
# Since trait_row is None, is_trait_available is False
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Skip this step since trait_row is None (clinical data for our trait of interest is not available)


False

### Step 3: Gene Data Extraction

In [4]:
# 1. Re-identify the SOFT and matrix files to ensure we have the correct paths
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract the gene expression data from the matrix file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers)
print("\nFirst 20 gene/probe identifiers:")
print(gene_data.index[:20])

# 4. Print the dimensions of the gene expression data
print(f"\nGene data dimensions: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")

# Note: we keep is_gene_available as True since we successfully extracted gene expression data
is_gene_available = True



First 20 gene/probe identifiers:
Index(['A1BG', 'A1CF', 'A2LD1', 'A2M', 'A2M.AS1', 'A2ML1', 'A2ML1.AS2',
       'A2MP1', 'A3GALT2P', 'A4GALT', 'A4GNT', 'AA06', 'AAAS', 'AACS',
       'AACSP1', 'AADAC', 'AADACL2', 'AADACL3', 'AADACL4', 'AADAT'],
      dtype='object', name='ID')

Gene data dimensions: 24421 genes × 58 samples


### Step 4: Gene Identifier Review

In [5]:
# These identifiers are human gene symbols, not probe IDs like Affymetrix probesets 
# that would require mapping.
# The identifiers like A1BG, A2M, AAAS, etc. are standard HGNC gene symbols.

requires_gene_mapping = False


### Step 5: Data Normalization and Linking

In [6]:
# 1. Normalize gene symbols in the index of gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
print(f"First 5 gene symbols after normalization: {normalized_gene_data.index[:5]}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Check if clinical data was properly loaded
# First, reload the clinical_data to make sure we're using the original data
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

# Print the sample IDs to understand the data structure
print("Sample IDs in clinical data:")
print(clinical_data.columns[:5], "...")  # Show first 5 sample IDs

# Print the sample IDs in gene expression data
print("Sample IDs in gene expression data:")
print(normalized_gene_data.columns[:5], "...")  # Show first 5 sample IDs

# Extract clinical features using the actual sample IDs
is_trait_available = trait_row is not None
linked_data = None

if is_trait_available:
    # Extract clinical features with proper sample IDs
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age if age_row is not None else None,
        gender_row=gender_row,
        convert_gender=convert_gender if gender_row is not None else None
    )
    
    print(f"Clinical data shape: {selected_clinical_df.shape}")
    print(f"Clinical data preview: {preview_df(selected_clinical_df, n=3)}")
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")
    
    # Link clinical and genetic data
    # Make sure both dataframes have compatible indices/columns
    linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
    print(f"Linked data shape before handling missing values: {linked_data.shape}")
    
    if linked_data.shape[0] == 0:
        print("WARNING: No samples matched between clinical and genetic data!")
        # Create a sample dataset for demonstration
        print("Using gene data with artificial trait values for demonstration")
        is_trait_available = False
        is_biased = True
        linked_data = pd.DataFrame(index=normalized_gene_data.columns)
        linked_data[trait] = 1  # Placeholder
    else:
        # 3. Handle missing values
        linked_data = handle_missing_values(linked_data, trait)
        print(f"Data shape after handling missing values: {linked_data.shape}")
        
        # 4. Determine if trait and demographic features are biased
        is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
        print(f"Data shape after removing biased features: {linked_data.shape}")
else:
    print("Trait data was determined to be unavailable in previous steps.")
    is_biased = True  # Set to True since we can't evaluate without trait data
    linked_data = pd.DataFrame(index=normalized_gene_data.columns)
    linked_data[trait] = 1  # Add a placeholder trait column
    print(f"Using placeholder data due to missing trait information, shape: {linked_data.shape}")

# 5. Validate and save cohort info
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note="Dataset contains gene expression data from multiple sclerosis patients, but there were issues linking clinical and genetic data."
)

# 6. Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable for associational studies.")

Gene data shape after normalization: (21521, 58)
First 5 gene symbols after normalization: Index(['A1BG', 'A1CF', 'A2M', 'A2ML1', 'A2MP1'], dtype='object', name='ID')


Normalized gene data saved to ../../output/preprocess/Amyotrophic_Lateral_Sclerosis/gene_data/GSE95810.csv
Sample IDs in clinical data:
Index(['!Sample_geo_accession', 'GSM2526327', 'GSM2526328', 'GSM2526329',
       'GSM2526330'],
      dtype='object') ...
Sample IDs in gene expression data:
Index(['GSM2526327', 'GSM2526328', 'GSM2526329', 'GSM2526330', 'GSM2526331'], dtype='object') ...
Trait data was determined to be unavailable in previous steps.
Using placeholder data due to missing trait information, shape: (58, 1)
Abnormality detected in the cohort: GSE95810. Preprocessing failed.
Dataset deemed not usable for associational studies.
